<a href="https://colab.research.google.com/github/maneeha/KGLLM/blob/main/Implement_RAG_with_Knowledge_Graph_and_Llama_Index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install llama_index pyvis Ipython langchain pypdf
!pip install llama-index-llms-huggingface
!pip install llama-index-embeddings-langchain

In [ ]:
llama-index==0.10.33
llama-index-core==0.10.33
llama-index-embeddings-langchain==0.1.2
llama-index-embeddings-openai==0.1.9
llama-index-legacy==0.9.48
llama-index-llms-huggingface==0.1.4
langchain==0.1.16
langchain-community==0.0.34
langchain-core==0.1.46

SyntaxError: invalid syntax (<ipython-input-2-f97bcfca1dba>, line 1)

## Concise overview of the Packages used
### SimpleDirectoryReader :
 to read unstructured data.
### ServiceContext :
Supples contextual data vital for orchestrating various services it has been deprecated and replaced by Settings
### KnowledgeGraphIndex :
Required for both construction and manipulation of Knowledge Graphs.
### SimpleGraphStore :
Serves as a straightforward repoistory for storing graph data.
### HuggingFaceInferenceAPI:
Modeule for leveraging Open-Source LLMs.
### Set up the LLM
here we have used Huggingface inference api endpoints

In [ ]:
import logging
import sys
#
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [ ]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import KnowledgeGraphIndex
from llama_index.core import Settings
from llama_index.core.graph_stores import SimpleGraphStore
from llama_index.core import StorageContext
from llama_index.llms.huggingface import HuggingFaceInferenceAPI
from langchain.embeddings import HuggingFaceEmbeddings
from llama_index.embeddings.langchain import LangchainEmbedding
from pyvis.network import Network

In [ ]:
from google.colab import userdata
HF_TOKEN=userdata.get('huggingfacellama7b')


In [ ]:

llm = HuggingFaceInferenceAPI(
    model_name="HuggingFaceH4/zephyr-7b-beta", token=HF_TOKEN)

In [ ]:
#from llama_index.embeddings import LangchainEmbedding
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings

embed_model = LangchainEmbedding(
  HuggingFaceInferenceAPIEmbeddings(api_key=HF_TOKEN,model_name="thenlper/gte-large")
)

In [ ]:
documents = SimpleDirectoryReader("").load_data()
print(len(documents))


ValueError: Directory /https://colab.research.google.com/notebooks/basic_features_overview.ipynb/ does not exist.

In [ ]:
#setup the service context (global setting of LLM)
Settings.llm = llm
Settings.chunk_size = 512

#setup the storage context
graph_store = SimpleGraphStore()
storage_context = StorageContext.from_defaults(graph_store=graph_store)

#Construct the Knowlege Graph Undex
index = KnowledgeGraphIndex.from_documents( documents=documents,
                                           max_triplets_per_chunk=3,
                                           storage_context=storage_context,
                                           embed_model=embed_model,
                                          include_embeddings=True)

In [ ]:
query = "What is ESOP?"
query_engine = index.as_query_engine(include_text=True,
                                     response_mode ="tree_summarize",
                                     embedding_mode="hybrid",
                                     similarity_top_k=5,)
#
message_template =f"""<|system|>Please check if the following pieces of context has any mention of the  keywords provided in the Question.If not then don't know the answer, just say that you don't know.Stop there.Please donot try to make up an answer.</s>
<|user|>
Question: {query}
Helpful Answer:
</s>"""
#
response = query_engine.query(message_template)
#
print(response.response.split("<|assistant|>")[-1].strip())

################OUTPUT#####################
ESOP stands for Employee Stock Ownership Plan. It is a type of employee benefit program in which a company offers its employees the opportunity to own company stock. ESOPs can be set up as a retirement plan or as a way for employees to purchase company stock at a discounted price. ESOPs require setup procedures, including obtaining shareholder and director approval, amending articles if necessary, reserving shares for issuance, drafting ESOP documents, and granting options to eligible employees. The taxation of ESOPs involves various considerations, including the taxation of the company's contributions to the plan, the taxation of employee distributions, and the taxation of company stock sold by the plan. ESOPs are considered an employee benefit because they provide employees with the opportunity to own a stake in the company they work for, which can lead to increased employee loyalty, motivation, and retention.


In [ ]:
#https://medium.aiplanet.com/implement-rag-with-knowledge-graph-and-llama-index-6a3370e93cdd